## 千问模型（qwen-plus）Assistant API案例

- [原文链接](https://help.aliyun.com/zh/model-studio/developer-reference/overview)

### 本文主要内容
1. Assistant API使用流程
2. 调用function call案例
3. 调用千问自带插件案例

## 0、环境准备

1、安装千问模型的SDK dashscope

In [1]:
!pip install dashscope

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


2、[阿里云百炼大模型服务平台](https://bailian.console.aliyun.com/#/home)注册账号，按照[教程](https://help.aliyun.com/zh/model-studio/getting-started/first-api-call-to-qwen)开通服务。

在`.env`文件中配置你的 `DASHSCOPE_API_KEY="YOUR_DASHSCOPE_API_KEY"`


通义千问-Plus模型有100w token的免费额度使用。

In [2]:
from http import HTTPStatus

import os
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import dashscope
dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")

from dashscope import Assistants, Threads, Messages, Runs

## 1、自定义函数

In [3]:
def big_add(left, right):
    return left + right

# 可以被回调的函数放入此字典
function_mapper = {
    "big_add": big_add,
}

## 2、创建assistant

In [4]:
def create_assistant():
    assistant = Assistants.create(
    model='qwen-plus',
    name='My helper',
    description='A tool helper.',
    instructions='You are a helpful assistant. When asked a question, use tools wherever possible.',
    # tool to use, current we support text to image text_to_image, quark_search, code_interpreter, and function call.
    tools=[
        {'type': "code_interpreter"},
        {'type': "generate_qrcode"},
#         {'type': "text_to_image"},
        {
        'type': 'function',
        'function': {
                'name': 'big_add',
                'description': 'Add to number',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'left': {
                            'type': 'integer',
                            'description': 'The left operator'
                        },
                        'right': {
                            'type': 'integer',
                            'description': 'The right operator.'
                        }
                    },
                    'required': ['left', 'right']
                }
        }
    }],)
    return assistant



assistant = create_assistant()

## 3、调用assistant执行流程

In [5]:
def send_message(assistant, message='你好'):
    print(f"Query: {message}")

    # 创建Thread
    thread = Threads.create()

    # 创建Message
    message = Messages.create(thread.id, content=message)
    
    # 创建Run
    run = Runs.create(thread.id, assistant_id=assistant.id)
#     print("=====Run======")
#     print(run)
    # 等待执行完成
    run_status = Runs.wait(run.id, thread_id=thread.id)

    # 如果需要调用function calling
    if run_status.required_action:
        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
        func_name = f['name']
        param = json.loads(f['arguments'])
        print("=====function======")
        print(f)
        if func_name in function_mapper:
            output = function_mapper[func_name](**param)
        else:
            output = ""

        tool_outputs = [{
            'output':
                str(output)
        }]
        print("===== tool output =====")
        print(tool_outputs)
        
        run = Runs.submit_tool_outputs(run.id,
                                       thread_id=thread.id,
                                       tool_outputs=tool_outputs)
#         print(run)
        run_status = Runs.wait(run.id, thread_id=thread.id)
    
    # get the thread messages.
    msgs = Messages.list(thread.id)
    
    print("运行结果:")
    for message in msgs['data'][::-1]:
        print(message)
        print("content: ", message['content'][0]['text']['value'])
    print("\n")

In [6]:
send_message(assistant=assistant,message="计算 87787 和 788988737的和")

Query: 计算 87787 和 788988737的和
{
    "assistant_id": "asst_6251335f-b055-4617-8c90-ead4caedcedc",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723044249433,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_45907766-853a-4a50-927f-63787372cc1c",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "69cc22c2-1650-9ec0-a5a2-92206d14f162",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_53b8cf09-ff77-4582-9c73-4762a52a6da6",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "descripti

{
    "assistant_id": "asst_6251335f-b055-4617-8c90-ead4caedcedc",
    "cancelled_at": null,
    "completed_at": 1723044251374,
    "created_at": 1723044249433,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_45907766-853a-4a50-927f-63787372cc1c",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "f7657b47-bd66-9e7b-805f-b2f69942f3d2",
    "required_action": {
        "submit_tool_outputs": {
            "tool_calls": [
                {
                    "function": {
                        "arguments": "{\"left\": 87787, \"right\": 788988737}",
                        "name": "big_add"
                    },
                    "id": "",
                    "type": "function"
                }
            ]
        },
        "type": "submit_tool_outputs"
 

In [7]:
send_message(assistant=assistant,message="我有一组数据，想让你帮我分析。以下是一个简单的列表：[4, 8, 15, 16, 23, 42]。请计算这些数值的平均值、中位数和方差，并告诉我结果。")

Query: 我有一组数据，想让你帮我分析。以下是一个简单的列表：[4, 8, 15, 16, 23, 42]。请计算这些数值的平均值、中位数和方差，并告诉我结果。
{
    "assistant_id": "asst_6251335f-b055-4617-8c90-ead4caedcedc",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723044255734,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_c28ae4ee-acef-41a9-899c-a129d2c24b60",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "3efd9020-6b45-90ae-a9ed-044e636cadd0",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_aec8e7cf-8c17-4489-a399-42f532c5fb03",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {

{
    "assistant_id": "asst_6251335f-b055-4617-8c90-ead4caedcedc",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723044255734,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_c28ae4ee-acef-41a9-899c-a129d2c24b60",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "08038fb2-c293-9b96-9cc8-289a65116db5",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_aec8e7cf-8c17-4489-a399-42f532c5fb03",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "description": "Add to number",
        

{
    "assistant_id": "asst_6251335f-b055-4617-8c90-ead4caedcedc",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723044255734,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_c28ae4ee-acef-41a9-899c-a129d2c24b60",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "49cf1a20-5be8-9317-8fa8-762494a9a11c",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_aec8e7cf-8c17-4489-a399-42f532c5fb03",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "description": "Add to number",
        

{
    "assistant_id": "asst_6251335f-b055-4617-8c90-ead4caedcedc",
    "cancelled_at": null,
    "completed_at": 1723044273597,
    "created_at": 1723044255734,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_c28ae4ee-acef-41a9-899c-a129d2c24b60",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "f12bc78c-d2c3-964d-8caa-847086bb5d54",
    "required_action": null,
    "started_at": null,
    "status": "completed",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_aec8e7cf-8c17-4489-a399-42f532c5fb03",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "description": "Add to numbe

In [8]:
send_message(assistant=assistant,message="将http://www.baidu.com转化为二维码")

Query: 将http://www.baidu.com转化为二维码
{
    "assistant_id": "asst_6251335f-b055-4617-8c90-ead4caedcedc",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723044274693,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_0d1ebd19-0609-49e7-97c1-734617d8b2ae",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "73d5fc52-e4ea-990c-8f39-a3b94c860481",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_140df7be-6255-42cf-b842-fca1a7b1d7c2",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "desc

{
    "assistant_id": "asst_6251335f-b055-4617-8c90-ead4caedcedc",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723044274693,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_0d1ebd19-0609-49e7-97c1-734617d8b2ae",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "a7b5c1e3-da25-9b09-87d0-78067245ac17",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_140df7be-6255-42cf-b842-fca1a7b1d7c2",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "description": "Add to number",
        